# Construction des features pour les données de télémétrie

In [1]:
import pandas as pd

On importe les données et on convertit le type de la colonne de temps.

In [2]:
df = pd.read_csv("../data/raw/PdM_telemetry.csv")
df["datetime"] = pd.to_datetime(df["datetime"])
df

,datetime,machineID,volt,rotate,pressure,vibration
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511
...,...,...,...,...,...,...
876095,2016-01-01 02:00:00,100,179.438162,395.222827,102.290715,50.771941
876096,2016-01-01 03:00:00,100,189.617555,446.207972,98.180607,35.123072
876097,2016-01-01 04:00:00,100,192.483414,447.816524,94.132837,48.314561
876098,2016-01-01 05:00:00,100,165.475310,413.771670,104.081073,44.835259


On divise la dataframe en créant une dataframe pour chaque machine.

In [3]:
df_by_machine = df.groupby("machineID")
n_machines = df["machineID"].nunique()
dfs = [] # liste contenant tous les dataframes
for i in range(1,n_machines+1):
    dfs.append(df_by_machine.get_group(i))

In [23]:
dfs[0].head(10) # machine 1

,datetime,machineID,volt,rotate,pressure,vibration
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511
5,2015-01-01 11:00:00,1,172.504839,430.323362,95.927042,35.655017
6,2015-01-01 12:00:00,1,156.556031,499.071623,111.755684,42.753920
7,2015-01-01 13:00:00,1,172.522781,409.624717,101.001083,35.482009
8,2015-01-01 14:00:00,1,175.324524,398.648781,110.624361,45.482287
9,2015-01-01 15:00:00,1,169.218423,460.850670,104.848230,39.901735


## Construction des features pour une machine

Pour chaque machine, on aggrége les données télémétriques par tranche de 24 heures toutes les 24 heures puis on calcule la moyenne et l'écart type.

1ère solution : on fait glisser la fenêtre et on enlève 23 entrées sur 24. -> pas optimisé

In [6]:
df_lag = dfs[0].rolling(window=24).agg([np.mean,np.std])
df_lag.head(30)

machineID             volt                 rotate              pressure  \
        mean  std        mean        std        mean        std       mean   
0        NaN  NaN         NaN        NaN         NaN        NaN        NaN   
1        NaN  NaN         NaN        NaN         NaN        NaN        NaN   
2        NaN  NaN         NaN        NaN         NaN        NaN        NaN   
3        NaN  NaN         NaN        NaN         NaN        NaN        NaN   
4        NaN  NaN         NaN        NaN         NaN        NaN        NaN   
5        NaN  NaN         NaN        NaN         NaN        NaN        NaN   
6        NaN  NaN         NaN        NaN         NaN        NaN        NaN   
7        NaN  NaN         NaN        NaN         NaN        NaN        NaN   
8        NaN  NaN         NaN        NaN         NaN        NaN        NaN   
9        NaN  NaN         NaN        NaN         NaN        NaN        NaN   
10       NaN  NaN         NaN        NaN         NaN        NaN        NaN   
11       NaN  NaN         NaN        NaN         NaN        NaN        NaN   
12       NaN  NaN         NaN        NaN         NaN        NaN        NaN   
13       NaN  NaN         NaN        NaN         NaN        NaN        NaN   
14       NaN  NaN         NaN        NaN         NaN        NaN        NaN   
15       NaN  NaN         NaN        NaN         NaN        NaN        NaN   
16       NaN  NaN         NaN        NaN         NaN        NaN        NaN   
17       NaN  NaN         NaN        NaN         NaN        NaN        NaN   
18       NaN  NaN         NaN        NaN         NaN        NaN        NaN   
19       NaN  NaN         NaN        NaN         NaN        NaN        NaN   
20       NaN  NaN         NaN        NaN         NaN        NaN        NaN   
21       NaN  NaN         NaN        NaN         NaN        NaN        NaN   
22       NaN  NaN         NaN        NaN         NaN        NaN        NaN   
23       1.0  0.0  169.733809  11.233120  445.179865  48.717395  96.797113   
24       1.0  0.0  170.614862  12.519402  446.364859  48.385076  96.849785   
25       1.0  0.0  170.961598  12.410607  445.610629  49.228285  97.179484   
26       1.0  0.0  170.525721  12.591948  443.906847  46.930282  97.667249   
27       1.0  0.0  169.893965  13.370357  447.009407  42.432317  97.715600   
28       1.0  0.0  170.012652  13.268830  446.075608  42.942968  96.833296   
29       1.0  0.0  170.049722  13.277336  446.461279  42.836780  96.906162   

               vibration            
          std       mean       std  
0         NaN        NaN       NaN  
1         NaN        NaN       NaN  
2         NaN        NaN       NaN  
3         NaN        NaN       NaN  
4         NaN        NaN       NaN  
5         NaN        NaN       NaN  
6         NaN        NaN       NaN  
7         NaN        NaN       NaN  
8         NaN        NaN       NaN  
9         NaN        NaN       NaN  
10        NaN        NaN       NaN  
11        NaN        NaN       NaN  
12        NaN        NaN       NaN  
13        NaN        NaN       NaN  
14        NaN        NaN       NaN  
15        NaN        NaN       NaN  
16        NaN        NaN       NaN  
17        NaN        NaN       NaN  
18        NaN        NaN       NaN  
19        NaN        NaN       NaN  
20        NaN        NaN       NaN  
21        NaN        NaN       NaN  
22        NaN        NaN       NaN  
23  10.079880  40.385160  5.853209  
24  10.171540  39.736826  6.163231  
25  10.252470  39.419434  6.161772  
26   9.406795  39.786670  6.098173  
27   9.471669  39.498374  6.195076  
28   9.072045  39.955593  5.523314  
29   9.071472  40.016513  5.481724

Une meilleure solution consisterait à grouper les valeurs par tranche de 24 heures avec pd.Grouper().

In [19]:
df_lag = dfs[0].groupby(pd.Grouper(freq="24h",key="datetime",closed="right",label="right",offset="6h")).agg([np.mean,np.std])
df_lag

machineID             volt                 rotate  \
                         mean  std        mean        std        mean   
datetime                                                                
2015-01-01 06:00:00         1  NaN  176.217853        NaN  418.504078   
2015-01-02 06:00:00         1  0.0  170.614862  12.519402  446.364859   
2015-01-03 06:00:00         1  0.0  169.533156  13.995465  454.785072   
2015-01-04 06:00:00         1  0.0  172.042428  12.848646  450.418764   
2015-01-05 06:00:00         1  0.0  171.929104  21.298322  443.448775   
...                       ...  ...         ...        ...         ...   
2015-12-28 06:00:00         1  0.0  172.334187  10.918948  446.826613   
2015-12-29 06:00:00         1  0.0  164.679670  13.838927  440.624526   
2015-12-30 06:00:00         1  0.0  165.026273  18.959558  454.785380   
2015-12-31 06:00:00         1  0.0  169.666008  14.300747  453.936163   
2016-01-01 06:00:00         1  0.0  170.444476  12.777815  447.836922   

                                  pressure             vibration            
                           std        mean        std       mean       std  
datetime                                                                    
2015-01-01 06:00:00        NaN  113.077935        NaN  45.087686       NaN  
2015-01-02 06:00:00  48.385076   96.849785  10.171540  39.736826  6.163231  
2015-01-03 06:00:00  40.843882  100.050567  11.036546  39.449734  5.561553  
2015-01-04 06:00:00  45.090576  100.284484   9.846662  52.153213  5.888262  
2015-01-05 06:00:00  36.054002   98.675590   9.885781  51.780445  5.246894  
...                        ...         ...        ...        ...       ...  
2015-12-28 06:00:00  41.869179  102.603659  10.197101  37.890590  5.484443  
2015-12-29 06:00:00  41.972165   99.784448  10.477541  40.399796  6.160479  
2015-12-30 06:00:00  64.404480   97.472731  10.763105  39.159190  4.916876  
2015-12-31 06:00:00  49.621917   99.130660   9.952543  39.937682  5.011262  
2016-01-01 06:00:00  61.927628   97.137329   9.148992  39.762388  5.305768  

[366 rows x 10 columns]

## Construction des features pour toutes les machines

On peut faire ce qu'on a fait précédemment pour toutes les machines en même temps avec :

In [24]:
lag_df = df.groupby(["machineID",pd.Grouper(freq="24h",key="datetime",closed="right",label="right",offset="6h")]).agg([np.mean,np.std])

In [28]:
lag_df.loc[100]

volt                 rotate               pressure  \
                           mean        std        mean        std        mean   
datetime                                                                        
2015-01-01 06:00:00  161.587466        NaN  399.879713        NaN  105.314528   
2015-01-02 06:00:00  170.493587  15.066954  440.275059  55.305847   98.398834   
2015-01-03 06:00:00  170.179565  15.628160  464.066801  47.433489   99.438607   
2015-01-04 06:00:00  166.418803  14.344253  457.958472  48.734608   99.682756   
2015-01-05 06:00:00  166.160006  19.825483  437.521043  46.118974   98.983478   
...                         ...        ...         ...        ...         ...   
2015-12-28 06:00:00  172.724209  15.068738  442.878518  57.950400   98.301343   
2015-12-29 06:00:00  170.078366  14.635941  465.577006  50.313046   99.858495   
2015-12-30 06:00:00  166.445547  14.414241  452.675501  59.222526   94.691789   
2015-12-31 06:00:00  166.902351  14.000620  460.641794  59.820047  101.634364   
2016-01-01 06:00:00  173.770428  15.573014  456.774345  61.859239  100.507221   

                                vibration            
                           std       mean       std  
datetime                                             
2015-01-01 06:00:00        NaN  38.559006       NaN  
2015-01-02 06:00:00  10.432241  40.462582  4.648896  
2015-01-03 06:00:00   8.536060  39.941947  4.655782  
2015-01-04 06:00:00   9.535573  40.005512  3.912343  
2015-01-05 06:00:00   8.392190  40.274948  4.431423  
...                        ...        ...       ...  
2015-12-28 06:00:00  11.207251  39.736106  5.519291  
2015-12-29 06:00:00   7.582133  39.499764  5.108806  
2015-12-30 06:00:00  10.569736  40.894142  4.693342  
2015-12-31 06:00:00   6.856021  39.346934  5.392136  
2016-01-01 06:00:00   9.942610  40.166266  6.191276  

[366 rows x 8 columns]

############ (C'est ce que j'ai fait pour 3h mais c'est aussi valable pour une fenêtre de 24h)

Chaque fenêtre est datée par la dernière date de la fenêtre : la moyenne pour "2015-01-01 09:00:00" correspond à la moyenne des données prises à 7, 8 et 9h.

```
      |     |     !  <---- LIGNES AGREGéES
6h    7h    8h    9h
|_____|_____|_____|  <---- FENêTRE CORRESPONDANTE 
```

Ici, la première date est "2016-01-01 06:00:00" et devrait être agrégée avec les données des deux entrées précédentes sauf qu'elles n'existent pas. Pandas a pris la valeurs des capteurs à 6h pour la moyenne et a mis NaN pour l'écart-type (il n'y a qu'une seule valeur disponible).
*--->* **Je propose de supprimer cette ligne pour chaque machine.**

##########

In [29]:
lag_df_flat = lag_df.reset_index() # indexation simple pour les index

lag_df_flat.columns = ['_'.join(col) if col[1]!="" else col[0] for col in lag_df_flat.columns.values] # indexation simple pour les colonnes

# On supprime la 1ère date pour chaque machine
first_datetime = lag_df_flat["datetime"].min()
lag_df_flat = lag_df_flat[lag_df_flat["datetime"] > first_datetime]

In [30]:
lag_df_flat[lag_df_flat["machineID"]==100]

,machineID,datetime,volt_mean,volt_std,rotate_mean,rotate_std,pressure_mean,pressure_std,vibration_mean,vibration_std
36235,100,2015-01-02 06:00:00,170.493587,15.066954,440.275059,55.305847,98.398834,10.432241,40.462582,4.648896
36236,100,2015-01-03 06:00:00,170.179565,15.628160,464.066801,47.433489,99.438607,8.536060,39.941947,4.655782
36237,100,2015-01-04 06:00:00,166.418803,14.344253,457.958472,48.734608,99.682756,9.535573,40.005512,3.912343
36238,100,2015-01-05 06:00:00,166.160006,19.825483,437.521043,46.118974,98.983478,8.392190,40.274948,4.431423
36239,100,2015-01-06 06:00:00,168.221073,15.268782,446.336273,46.152941,100.695469,10.560879,42.165682,5.993114
...,...,...,...,...,...,...,...,...,...,...
36595,100,2015-12-28 06:00:00,172.724209,15.068738,442.878518,57.950400,98.301343,11.207251,39.736106,5.519291
36596,100,2015-12-29 06:00:00,170.078366,14.635941,465.577006,50.313046,99.858495,7.582133,39.499764,5.108806
36597,100,2015-12-30 06:00:00,166.445547,14.414241,452.675501,59.222526,94.691789,10.569736,40.894142,4.693342
36598,100,2015-12-31 06:00:00,166.902351,14.000620,460.641794,59.820047,101.634364,6.856021,39.346934,5.392136


In [31]:
def transform_telemetry():
    """Transforme les données télémétriques.
    
    """
    #
    df = pd.read_csv("../data/raw/PdM_telemetry.csv")
    df["datetime"] = pd.to_datetime(df["datetime"])
    #
    lag_df = df.groupby(["machineID",pd.Grouper(freq="24h",key="datetime",closed="right",label="right",offset="6h")]).agg([np.mean,np.std])
    lag_df_flat = lag_df.reset_index() # indexation simple pour les index
    lag_df_flat.columns = ['_'.join(col) if col[1]!="" else col[0] for col in lag_df_flat.columns.values] # indexation simple pour les colonnes
    # On supprime la 1ère date pour chaque machine
    first_datetime = lag_df_flat["datetime"].min()
    lag_df_flat = lag_df_flat[lag_df_flat["datetime"] > first_datetime]
    lag_df_flat.to_csv("../data/prepared_data/telemetrie.csv",index=False)

In [32]:
transform_telemetry()

############ (C'est ce que j'ai fait pour 3h mais c'est aussi valable pour une fenêtre de 24h)

## Alternative pour le calcul des features avec une fenêtre glissante

Si on veut faire une fenêtre glissante de 3 heures pour toutes les heures(plus long à calculer) :

In [18]:
lag_df2 = df.groupby("machineID").rolling(window=3).agg([np.mean,np.std])
lag_df2.loc[100]

machineID                      volt                 rotate             \
            mean           std        mean        std        mean        std   
867339       NaN           NaN         NaN        NaN         NaN        NaN   
867340       NaN           NaN         NaN        NaN         NaN        NaN   
867341     100.0  2.264780e-07  159.615566   5.003975  410.116312  22.587648   
867342     100.0  2.264780e-07  163.970945  10.378482  456.323216  74.137920   
867343     100.0  2.264780e-07  176.072401  22.888746  478.272366  53.555110   
...          ...           ...         ...        ...         ...        ...   
876095     100.0  2.264780e-07  170.306956   8.457322  438.653616  43.021553   
876096     100.0  2.264780e-07  177.266129  13.568463  440.894903  43.260932   
876097     100.0  2.264780e-07  187.179710   6.855798  429.749108  29.911451   
876098     100.0  2.264780e-07  182.525427  14.835200  435.932056  19.208302   
876099     100.0  2.264780e-07  176.431587  14.206791  452.561688  41.367222   

          pressure             vibration            
              mean        std       mean       std  
867339         NaN        NaN        NaN       NaN  
867340         NaN        NaN        NaN       NaN  
867341   96.545326  15.380333  39.413594  2.947742  
867342   98.078753  16.850804  41.602041  4.176891  
867343   91.740625  16.207489  44.026158  1.231817  
...            ...        ...        ...       ...  
876095  102.346227   0.785721  47.481232  3.198928  
876096  100.687019   2.198731  43.425922  7.868190  
876097   98.201386   4.078979  44.736524  8.415672  
876098   98.798172   5.002788  42.757630  6.836757  
876099   92.436483  12.578849  43.665022  5.331860  

[8761 rows x 10 columns]